# Notes and Cheatsheet

This is a summary of all the lessons learned so far.

In order to run applications we write in TypeScript, we need to compile the TypeScript files into JavaScript. We can either do this once with the `tsc` command, or we can watch (recursively) the current working directory with `tsc --watch` or `tsc -w`.

In [1]:
// import the typechecker function so that we can demo type checking inline
import { checkTypeScript } from './utils/typechecker.ts';


## A note on type checking

TypeScript does not prevent us from running code that will error at runtime.


In [2]:
const double = (x: number): number => x * 2;
double('anything');


NaN

We're still able to call `double('anything')` and get `NaN` in response, just as we would with JavaScript. It's only when we compile or use a typechecker (as we do with the `checkTypeScript` function) that we see the error for a non-number argument value.


In [3]:
await checkTypeScript(`const double = (x: number): number => x * 2; double('anything');`);

error: TS2345 [ERROR]: Argument of type 'string' is not assignable to parameter of type 'number'.
const double = (x: number): number => x * 2; double('anything');
                                                    ~~~~~~~~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/490a358d49f7ebda.ts:1:53


Our IDE may also provide type hints to help us.

## Variables

A variable has a type. The type is separated from the variable name with `:`.

In [4]:
// type annotations
let sum: number = 1 + 2;
sum;

3

In [5]:
// check the type of sum
await checkTypeScript(`let sum: number = 1 + 2;`);


Types are correct.


In [6]:
// type annotations
let sum: number = 'any' + 'thing';
sum;

"anything"

In [7]:
// type errors
await checkTypeScript(`let sum: number = 'any' + 'thing';`);


error: TS2322 [ERROR]: Type 'string' is not assignable to type 'number'.
let sum: number = 'any' + 'thing';
    ~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/4dce83785aa19d53.ts:1:5


## Functions

A function takes arguments, which have types. The function returns something, which also has a type. The types are marked with `:`, as they were for simple variables.

In [8]:
function add(a: number, b: number): number {
  return a + b;
}

In [9]:
add(1, 2);


3

In [10]:
await checkTypeScript(`function add(a: number, b: number): number { return a + b; }; add(1, 2);`);


Types are correct.


**NOTE**: if we don't set a type explicitly for our params then they will default to `any`.

**NOTE**: we need to declare the type for each param, even if two or more share the same type. For example, the following will make the type of `a` equal to `any` _not_ `number`:

In [11]:
await checkTypeScript(`function add(a, b: number): number { return a + b; }; add(1, 2);`);


error: TS7006 [ERROR]: Parameter 'a' implicitly has an 'any' type.
function add(a, b: number): number { return a + b; }; add(1, 2);
             ^
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/7c792f74c0c56975.ts:1:14


### This is what TypeScript was designed for

If we pass `true` in to the add function it appears to work, we get the number `2` as a result, but this is because JavaScript implicitly converts `true` to `1` (and `false` to `0`).

In [12]:
add(1, true)  // true is implicitly converted to 1 by the JavaScript runtime

2

However, with the type checker (or compiler), will return an error because we're trying to use a boolean instead of a number.

In [13]:
await checkTypeScript(`function add(a: number, b: number): number { return a + b; }; add(1, true);`);


error: TS2345 [ERROR]: Argument of type 'boolean' is not assignable to parameter of type 'number'.
function add(a: number, b: number): number { return a + b; }; add(1, true);
                                                                     ~~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/1be0c3842f203e54.ts:1:70


## Operators


In [14]:
// these should run just fine
console.log(1 + 1);
console.log(2 * 2);
console.log('1' + '1'); // TypeScript allows string concatenation with `+`

// NOTE: TypeScript will also allow mixed concatenation of strings and numbers
console.log('1' + 1);
console.log(1 + '1');

// these should throw type errors but will "work" at runtime
console.log('2' * '2');
console.log('4' / '2');
console.log(1 / '1');
console.log('1' / 1);


2
4
11
11
11
4
2
1
1


In [15]:
// these should check just fine
await checkTypeScript(`console.log(1 + 1);`);
await checkTypeScript(`console.log(2 * 2);`);
await checkTypeScript(`console.log('1' + '1');`); // TypeScript allows string concatenation with `+`

// NOTE: TypeScript will also allow mixed concatenation of strings and numbers
await checkTypeScript(`console.log('1' + 1);`);
await checkTypeScript(`console.log(1 + '1');`);

// these should throw type errors
await checkTypeScript(`console.log('2' * '2');`);
await checkTypeScript(`console.log('4' / '2');`);
await checkTypeScript(`console.log(1 / '1');`);
await checkTypeScript(`console.log('1' / 1);`);


Types are correct.
Types are correct.
Types are correct.
Types are correct.
Types are correct.
error: TS2362 [ERROR]: The left-hand side of an arithmetic operation must be of type 'any', 'number', 'bigint' or an enum type.
console.log('2' * '2');
            ~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/5562fa0de1c3ef64.ts:1:13

TS2363 [ERROR]: The right-hand side of an arithmetic operation must be of type 'any', 'number', 'bigint' or an enum type.
console.log('2' * '2');
                  ~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/5562fa0de1c3ef64.ts:1:19

Found 2 errors.
error: TS2362 [ERROR]: The left-hand side of an arithmetic operation must be of type 'any', 'number', 'bigint' or an enum type.
console.log('4' / '2');
            ~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/3779d27f424526eb.ts:1:13

TS2363 [ERROR]: The right-hand side of an arithmetic operation must be of type 'any', 'number', 'bigint' or an enum ty

# Types

Types can be defined using the `type` keyword. Types are idiomatically named using `UpperCamelCase`.

In [18]:
type MyStringType = string;
let myString: MyStringType = 'Hello, world!';
myString;

"Hello, world!"

## Type Erasure

The TypeScript compiler works by reading through a TypeScript file and looking for any type annotations. It checks any types it finds and then **removes the TypeScript code** turning it into plain JavaScript.

This means that our runtime code contains no TypeScript, so we can't do things like inspect types at runtime.